In [20]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.callbacks import ModelCheckpoint

In [8]:
import os
import random
from shutil import copyfile

# Set the paths for the original dataset and the new directories
original_dir = '/kaggle/input/diabetic-retinopathy-2015-data-colored-resized/colored_images/colored_images'
train_dir = '/kaggle/working/train'
test_dir = '/kaggle/working/test'

# Set the classes of diabetic retinopathy
classes = ['No_DR', 'Mild', 'Moderate', 'Severe', 'Proliferate_DR']

# Create the new directories if they don't exist
if not os.path.exists(train_dir):
    os.makedirs(train_dir)

if not os.path.exists(test_dir):
    os.makedirs(test_dir)

# Set the train-test split ratio
split_ratio = 0.8

# Iterate over the classes and copy the files to the train or test directories
for c in classes:
    # Set the paths for the class folder in the original dataset and the new directories
    class_original_dir = os.path.join(original_dir, c)
    class_train_dir = os.path.join(train_dir, c)
    class_test_dir = os.path.join(test_dir, c)

    # Create the class folders in the new directories if they don't exist
    if not os.path.exists(class_train_dir):
        os.makedirs(class_train_dir)

    if not os.path.exists(class_test_dir):
        os.makedirs(class_test_dir)

    # Get the list of files in the class folder
    files = os.listdir(class_original_dir)

    # Shuffle the files randomly
    random.shuffle(files)

    # Split the files between the train and test directories
    split_index = int(split_ratio * len(files))
    train_files = files[:split_index]
    test_files = files[split_index:]

    # Copy the train files to the train directory
    for f in train_files:
        src = os.path.join(class_original_dir, f)
        dst = os.path.join(class_train_dir, f)
        copyfile(src, dst)

    # Copy the test files to the test directory
    for f in test_files:
        src = os.path.join(class_original_dir, f)
        dst = os.path.join(class_test_dir, f)
        copyfile(src, dst)


In [10]:
def preprocess_image(img):
    img = exposure.equalize_adapthist(img, clip_limit=0.03)
    
    
    return img

In [11]:
train_dir = '/kaggle/working/train'
test_dir = '/kaggle/working/test'

In [28]:
img_size = 255
BATCH_SIZE = 32

In [58]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   )
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(224, 224),
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='categorical')

Found 28099 images belonging to 5 classes.


In [59]:
test_datagen = ImageDataGenerator(rescale=1./255,
                                 )
test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(224, 224),
                                                  batch_size=BATCH_SIZE,
                                                  class_mode='categorical')

Found 7027 images belonging to 5 classes.


In [60]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [61]:
from tensorflow.keras.models import Sequential
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of the base model
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [62]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // BATCH_SIZE,
                    epochs=10,
                    validation_data=test_generator,
                    validation_steps=test_generator.samples // BATCH_SIZE)

# Evaluate the model on the testing set
loss, accuracy = model.evaluate(test_generator)
print('Loss:', loss)
print('Accuracy:', accuracy)


Epoch 1/10
343/878 [==========>...................] - ETA: 22:28 - loss: 4.5697 - accuracy: 0.6946